# Import software libraries

In [ ]:
import sys                                                  # Read system parameters.
import pandas as pd                                         # Manipulate and analyze data.
import matplotlib                                           # Create and format charts.
import matplotlib.pyplot as plt
import seaborn as sns                                       # Make charting easier.
import yellowbrick                                          # Visualize elbow and silhouette plots.
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
import sklearn                                              # Train and evaluate machine learning models.
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from collections import Counter                             # Count objects in containers.
import pickle                                               # Save Python objects as binary files.
import warnings                                             # Suppress warnings.
warnings.filterwarnings('ignore')

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- Matplotlib {}'.format(matplotlib.__version__))
print('- Seaborn {}'.format(sns.__version__))
print('- Yellowbrick {}'.format(yellowbrick.__version__))
print('- scikit-learn {}'.format(sklearn.__version__))

# Load and preview the data

In [ ]:
users_data = pd.read_pickle('/home/student/CDSP/Clustering/data/users_data_final.pickle')

users_data.head(n = 5)

# Check the shape of the data

In [ ]:
users_data.shape

# Check the data types

In [ ]:
users_data.info()

# Filter by demographics data

In [ ]:
users_data_demographics = \
users_data.filter(regex = 'education|job|age|single')

users_data_demographics.head(n = 3)

# Scale the data

In [ ]:
users_data_demographics.describe()

In [ ]:
# Standardize the data.

scaler = StandardScaler()
        
scaler.fit(users_data_demographics)
users_data_scaled = scaler.transform(users_data_demographics)

print('New standard deviation: ', users_data_scaled.std())
print('New mean:               ', round(users_data_scaled.mean()))

# Train a ***k***-means clustering model

In [ ]:
# Specify initial number of clusters.

n_clusters = 5

In [ ]:
# Build k-means model.

kmeans = KMeans(n_clusters = n_clusters, random_state = 10)

# Fit scaled data to model.

kmeans.fit(users_data_scaled)

# Generate the clusters

In [ ]:
y_kmeans = kmeans.predict(users_data_scaled)

results = pd.DataFrame(users_data_demographics)
results.insert(0, 'cluster', y_kmeans)
results.head()

# Visualize the number of users in each cluster

In [ ]:
def cluster_bar(cluster_labels):
    """Create a bar chart to show number of users in each cluster."""
    pd.DataFrame(Counter(cluster_labels).most_common()). \
    set_index(0).plot.bar(legend = None)
    
    plt.title('Distribution of Clusters')
    plt.xlabel('Cluster ID')
    plt.xticks(rotation = 0)
    plt.ylabel('Number of users in cluster');

In [ ]:
cluster_bar(y_kmeans)

# Filter the data by education and scale it

In [ ]:
users_data_reduced = users_data.filter(regex = 'education')
users_data_reduced.head(n = 3)

In [ ]:
# Standardize the data.

scaler = StandardScaler()
        
scaler.fit(users_data_reduced)
users_data_reduced_scaled = scaler.transform(users_data_reduced)

# Train a hierarchical clustering model

In [ ]:
agglom = AgglomerativeClustering(n_clusters = n_clusters,
                                 affinity = 'euclidean',
                                 linkage = 'single')

agglom.fit(users_data_reduced_scaled)
y_agglom = agglom.fit_predict(users_data_reduced_scaled)

results['cluster'] = y_agglom
results.head()

In [ ]:
cluster_bar(y_agglom)

# Adjust the number of clusters in the ***k***-means clustering model

In [ ]:
n_clusters = 10

kmeans = KMeans(n_clusters = n_clusters, random_state = 10)

kmeans.fit(users_data_scaled)

# Examine the results

In [ ]:
y_kmeans = kmeans.predict(users_data_scaled)

results['cluster'] = y_kmeans
results.head()

In [ ]:
cluster_bar(y_kmeans)

# Load the PCA-reduced demographics data

In [ ]:
pca_df = pd.read_pickle('/home/student/CDSP/Clustering/data/users_data_demo_pca.pickle')

pca_df = pd.concat([pca_df, pd.DataFrame(y_kmeans)], axis = 1). \
rename(columns = {0: 'cluster'})

pca_df

# Visualize the clusters on the reduced data

In [ ]:
cmap = sns.color_palette('tab10', n_colors = n_clusters, desat = .5)

sns.scatterplot(x = 'PCA1', y = 'PCA2',
                hue = 'cluster', data = pca_df[::25],
                palette = cmap, legend = True)

plt.title('k-Means Clustering with 2 Dimensions');

# Use the elbow method to determine the optimal number of clusters

In [ ]:
elbow = KElbowVisualizer(kmeans, k = (1, 20))
elbow.fit(users_data_scaled)
elbow.poof();

# Use silhouette analysis to determine the optimal number of clusters

In [ ]:
silhouette = SilhouetteVisualizer(KMeans(12, random_state = 10))
silhouette.fit(users_data_scaled)
silhouette.poof();

In [ ]:
print('Number of clusters: ', silhouette.n_clusters_)
print('Silhouette score:   ', silhouette.silhouette_score_)

# Try a different number of clusters and compare the silhouette scores

In [ ]:
silhouette = SilhouetteVisualizer(KMeans(20, random_state = 10))
silhouette.fit(users_data_scaled)
silhouette.poof();

In [ ]:
print('Number of clusters: ', silhouette.n_clusters_)
print('Silhouette score:   ', silhouette.silhouette_score_)

# Retrain the ***k***-means model on the optimal number of clusters

In [ ]:
n_clusters = 12

kmeans = KMeans(n_clusters = n_clusters, random_state = 10)

kmeans.fit(users_data_scaled)

# Generate the clusters and the number of users in each cluster

In [ ]:
y_kmeans = kmeans.predict(users_data_scaled)

results['cluster'] = y_kmeans
results.head()

In [ ]:
cluster_bar(y_kmeans)

# Compare two different clusters

In [ ]:
cluster_0 = results[results.cluster == 0]
cluster_0.describe()

In [ ]:
sns.countplot(cluster_0, x = 'age_group_encoded');

In [ ]:
cluster_6 = results[results.cluster == 6]
cluster_6.describe()

In [ ]:
sns.countplot(cluster_6, x = 'age_group_encoded');

# Save the best model

In [ ]:
pickle.dump(kmeans, open('kmeans.pickle', 'wb'))